# 🤙 Comfyui on NVIDIA Brev

<div style="background: linear-gradient(90deg, #00ff87 0%, #60efff 100%); padding: 1px; border-radius: 8px; margin: 20px 0;">
    <div style="background: #0a0a0a; padding: 20px; border-radius: 7px;">
        <p style="color: #60efff; margin: 0;"><strong>⚡ Powered by Brev</strong> | Converted from <a href="https://github.com/unslothai/notebooks/blob/main/nb/comfyui.ipynb" style="color: #00ff87;">Unsloth Notebook</a></p>
    </div>
</div>

## 📋 Configuration

<div style="text-align: left;">

| Parameter | Value |
|:----------|:------|
| **Model** | Comfyui |
| **Recommended GPU** | L4 |
| **Min VRAM** | 16 GB |
| **Batch Size** | 2 |
| **Categories** | fine-tuning |

</div>

## 🔧 Key Adaptations for Brev

- ✅ Replaced Colab-specific installation with conda-based Unsloth
- ✅ Converted magic commands to subprocess calls
- ✅ Removed Google Drive dependencies
- ✅ Updated paths from `/workspace/` to `/workspace/`
- ✅ Added `device_map="auto"` for multi-GPU support
- ✅ Optimized batch sizes for NVIDIA GPUs

## 📚 Resources

- [Unsloth Documentation](https://docs.unsloth.ai/)
- [Brev Documentation](https://docs.nvidia.com/brev)
- [Original Notebook](https://github.com/unslothai/notebooks/blob/main/nb/comfyui.ipynb)


# Launch ComfyUI on Brev!

Welcome!

In this notebook, we have adapted ComfyUI's default Colab notebook to run on Brev. ComfyUI is a powerful GUI for Stable Diffusion models. It uses a node-based architecture that allows users to construct very complex image/video generation workflows by connecting different nodes, each representing a different function or operation!

In [ ]:
# Environment Setup

from pathlib import Path

OPTIONS = {}

WORKSPACE = 'ComfyUI'
USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['USE_COMFYUI_MANAGER'] = USE_COMFYUI_MANAGER
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

In [ ]:
%%bash
if ! [ -d WORKSPACE ]; then
    echo "Starting initial setup of ComfyUI..."
    git clone https://github.com/comfyanonymous/ComfyUI
    if [ -d ComfyUI ]; then
        echo "Repository cloned successfully into 'ComfyUI'."
        cd ComfyUI
    else
        echo "Failed to clone repository."
    fi
else
    echo "Workspace directory already exists."
fi

In [ ]:
import subprocess
import sys

# Check and handle updates for ComfyUI
if OPTIONS['UPDATE_COMFY_UI']:
    subprocess.run(['echo "Updating ComfyUI..."'], check=True, shell=True)
    subprocess.run(['git pull'], check=True, shell=True)

subprocess.run(['echo "Installing dependencies..."'], check=True, shell=True)
subprocess.check_call([sys.executable, "-m", "pip", "install", 'xformers!=0.0.18 -r ComfyUI/requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117'])

In [ ]:
import subprocess
import sys

import os

# Check if we should use ComfyUI Manager
if 'USE_COMFYUI_MANAGER' in OPTIONS and OPTIONS['USE_COMFYUI_MANAGER']:
    # Change directory to custom_nodes if it exists
    %cd custom_nodes

    # Ensure executable permissions for various scripts
    if not os.path.isfile("ComfyUI-Manager/check.sh"):
        subprocess.run(['chmod 755 ComfyUI-Manager/check.sh'], check=True, shell=True)
    if not os.path.isfile("ComfyUI-Manager/scan.sh"):
        subprocess.run(['chmod 755 ComfyUI-Manager/scan.sh'], check=True, shell=True)
    if not os.path.isfile("ComfyUI-Manager/node_db/dev/scan.sh"):
        subprocess.run(['chmod 755 ComfyUI-Manager/node_db/dev/scan.sh'], check=True, shell=True)
    if not os.path.isfile("ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh"):
        subprocess.run(['chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh'], check=True, shell=True)
    if not os.path.isfile("ComfyUI-Manager/scripts/install-comfyui-venv-win.bat"):
        subprocess.run(['chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-win.bat'], check=True, shell=True)

    # Clone the ComfyUI Manager repository if it does not exist
    if not os.path.isdir("ComfyUI-Manager"):
        print("-= Initial setup ComfyUI-Manager =-")
        subprocess.run(['git clone https://github.com/ltdrdata/ComfyUI-Manager /home/ubuntu/verb-workspace/ComfyUI/custom_nodes'], check=True, shell=True)

    # Change directory to ComfyUI-Manager and pull the latest changes
    %cd ComfyUI-Manager
    subprocess.run(['git pull'], check=True, shell=True)

    # Navigate back to the initial directory
    %cd ..

# Return to the workspace directory
%cd $WORKSPACE

# Install dependencies for custom nodes if required
if 'INSTALL_CUSTOM_NODES_DEPENDENCIES' in OPTIONS and OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
    print("-= Install custom nodes dependencies =-")
    subprocess.check_call([sys.executable, "-m", "pip", "install", 'GitPython'])
    subprocess.run(['python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies'], check=True, shell=True)

In [ ]:
import subprocess
import sys

import os
subprocess.check_call([sys.executable, "-m", "pip", "install", 'huggingface_hub'])

In [ ]:
import subprocess
import sys

subprocess.run(['wget -c https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -P ./models/checkpoints/'], check=True, shell=True)

In [ ]:
import subprocess
import sys

subprocess.run(['wget -c https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -P ./models/vae/'], check=True, shell=True)

In [ ]:
import subprocess
import threading
import time
import socket

def check_server(port):
    while True:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            result = sock.connect_ex(('127.0.0.1', port))
            if result == 0:
                print("\nServer is up and running at http://129.80.69.99:{}".format(port))
                break
            time.sleep(0.5)

threading.Thread(target=check_server, daemon=True, args=(8188,)).start()

In [ ]:
import subprocess
import sys

subprocess.run(['python main.py --listen 0.0.0.0 --port 8188 --dont-print-server --disable-xformers'], check=True, shell=True)

In [ ]:
def check_server(port):
    while True:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            result = sock.connect_ex(('127.0.0.1', port))
            if result == 0:
                print("\nServer is up and running at http://129.80.69.99:{}".format(port))
                break
            time.sleep(0.5)

threading.Thread(target=check_server, daemon=True, args=(8188,)).start()